In [128]:
import pandas as pd

#load and preview data

results = pd.read_csv('results.csv')
goalscorers = pd.read_csv('goalscorers.csv')
shootouts = pd.read_csv('shootouts.csv')

print(results.head())
print(goalscorers.head())
print(shootouts.head())

         date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  neutral  
0  Scotland    False  
1   England    False  
2  Scotland    False  
3   England    False  
4  Scotland    False  
         date  home_team away_team       team            scorer  minute  \
0  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    44.0   
1  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    55.0   
2  1916-07-02      Chile   Uruguay    Uruguay  Isabelino Gradín    70.0   
3  1916-07-02      Chile   Uruguay    Uruguay   José Piendibene    75.0   


In [129]:
#average number of goals per game between 1900 and 2000

#goals per match
results['total_goals'] = results['home_score'] + results['away_score']

#games between 1900 and 2000
results['year'] = results['date'].str[:4].astype(int)
matches_1900_2000 = results[(results['year'] >= 1900) & (results['year'] <= 2000)]

#average goals per match
avg_goals = matches_1900_2000['total_goals'].mean()
print(f"Average goals per match between 1900-2000: {avg_goals}")

Average goals per match between 1900-2000: 3.0704284750337383


In [130]:
#number of shootouts wins by country arranged in alphabetical order

shootout_wins = shootouts.groupby('winner').size().reset_index(name='wins').sort_values('winner')

shootout_wins

,winner,wins
0,Abkhazia,2
1,Algeria,7
2,Angola,7
3,Antigua and Barbuda,2
4,Argentina,14
...,...,...
158,Zambia,13
159,Zanzibar,2
160,Zimbabwe,7
161,Åland,1


In [131]:
# date column into datetime format
results['date'] = pd.to_datetime(results['date'])
goalscorers['date'] = pd.to_datetime(goalscorers['date'])
shootouts['date'] = pd.to_datetime(shootouts['date'])

# composite key; using date, home team, away team

results['key'] = results['date'].dt.strftime('%Y-%m-%d') + "_" + results['home_team'] + "_" + results['away_team']
goalscorers['key'] = goalscorers['date'].dt.strftime('%Y-%m-%d') + "_" + goalscorers['home_team'] + "_" + goalscorers['away_team']
shootouts['key'] = shootouts['date'].dt.strftime('%Y-%m-%d') + "_" + shootouts['home_team'] + "_" + shootouts['away_team']

In [132]:
# which teams won shootout after 1-1 draw

# since scores and shootout outcomes are stored in separtae files, first we merge
merged = results.merge(shootouts, on='key', how='inner')
# filter for 1-1 draws
one_on_one_draws = merged[(merged['home_score'] == 1) & (merged['away_score'] == 1)]
# winning teams
winners_after_draw = one_on_one_draws[['winner']].drop_duplicates().reset_index(drop=True)

winners_after_draw

,winner
0,Taiwan
1,South Korea
2,Guinea
3,Mauritius
4,Singapore
...,...
110,Iceland
111,Saint Kitts and Nevis
112,Guyana
113,India


In [133]:
# top goal scorer in tournament and what percent of goals are theirs

# the tournament data and score data is stored in results and goalscorers so we merge them
goal_tournament_merged = goalscorers.merge(results[['key','tournament', 'home_score', 'away_score']], on='key', how='left')

# goals per player per tournament
player_totals = (goal_tournament_merged.groupby(['tournament', 'scorer']).size().reset_index(name='goals_scored'))

# total goals per tournament
tournament_totals = (goal_tournament_merged.groupby('tournament').size().reset_index(name='total_goals'))

# merged for percent
merged = player_totals.merge(tournament_totals, on='tournament')
merged['percentage'] = (merged['goals_scored'] / merged['total_goals']) * 100

# top scorer per tournament
top_scorers = merged.sort_values(['tournament', 'goals_scored'], ascending=[True, False]).groupby('tournament').head(1).reset_index(drop=True)

top_scorers[['tournament', 'scorer', 'goals_scored', 'percentage']]

,tournament,scorer,goals_scored,percentage
0,AFC Asian Cup,Ali Daei,14,1.415571
1,African Cup of Nations,Samuel Eto'o,18,1.018676
2,Baltic Cup,Ēriks Pētersons,9,3.930131
3,British Home Championship,Geoff Hurst,4,12.121212
4,CONMEBOL–UEFA Cup of Champions,Claudio Caniggia,1,14.285714
5,Confederations Cup,Cuauhtémoc Blanco,9,2.127660
6,Copa América,Norberto Doroteo Méndez,17,0.636466
7,FIFA World Cup,Miroslav Klose,16,0.588235
8,FIFA World Cup qualification,Carlos Ruiz,39,0.171519
9,Gold Cup,Landon Donovan,18,1.640839


In [134]:
# data quality issues removed from results  
results['data_quality_flag'] = (results['home_team'].isna() | results['away_team'].isna() | results['tournament'].isna() | results['date'].isna() | results['home_score'].isna() | results['away_score'].isna() | (results['home_score'] < 0) | (results['away_score'] < 0))
results_clean = results[~results['data_quality_flag']].copy()


# data quality issues removed from goalscorers
goalscorers['data_quality_flag'] = (goalscorers['team'].isna() | goalscorers['scorer'].isna() | goalscorers['date'].isna() | (goalscorers['minute'] < 0) | (goalscorers['minute'] > 120))
goalscorers_clean = goalscorers[~goalscorers['data_quality_flag']].copy()

# data quality issues removed rom shootouts
shootouts['data_quality_flag'] = (shootouts['winner'].isna() | shootouts['home_team'].isna() | shootouts['away_team'].isna() | shootouts['date'].isna())
shootouts_clean =  shootouts[~shootouts['data_quality_flag']].copy()

# quality of each dataset before cleaning
display(results['data_quality_flag'].value_counts())
display(goalscorers['data_quality_flag'].value_counts())
display(shootouts['data_quality_flag'].value_counts())

# quality of each dataset after cleaning
display(results_clean['data_quality_flag'].value_counts())
display(goalscorers_clean['data_quality_flag'].value_counts())
display(shootouts_clean['data_quality_flag'].value_counts())

data_quality_flag
False    45315
Name: count, dtype: int64

data_quality_flag
False    43138
True        51
Name: count, dtype: int64

data_quality_flag
False    562
Name: count, dtype: int64

data_quality_flag
False    45315
Name: count, dtype: int64

data_quality_flag
False    43138
Name: count, dtype: int64

data_quality_flag
False    562
Name: count, dtype: int64